# Extraction, Analyse de Sentiment et Insertion en Base des Articles GDELT
Ce notebook récupère des articles via GDELT, calcule leur sentiment avec VADER, et les insère dans une base MariaDB ou SQLite.

In [3]:
# pip install gdeltdoc vaderSentiment beautifulsoup4 sqlalchemy pymysql
import pandas as pd
import re, html
import hashlib
from bs4 import BeautifulSoup
from datetime import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from gdeltdoc import GdeltDoc, Filters, repeat

In [2]:
USE_MARIADB = True

from sqlalchemy import create_engine, text

USER = "root"
PWD  = "2003"
HOST = "127.0.0.1"
PORT = 3306
DB   = "NewsVader"

if USE_MARIADB:
    ENGINE_URL = f"mysql+pymysql://{USER}:{PWD}@{HOST}:{PORT}/{DB}?charset=utf8mb4"
else:
    ENGINE_URL = "sqlite:///NewsVader.db"

# Crée la base si elle n'existe pas (MariaDB)
if USE_MARIADB:
    ADMIN_URL = f"mysql+pymysql://{USER}:{PWD}@{HOST}:{PORT}/?charset=utf8mb4"
    admin_engine = create_engine(ADMIN_URL, future=True, pool_pre_ping=True)
    with admin_engine.begin() as conn:
        conn.exec_driver_sql(f"""
            CREATE DATABASE IF NOT EXISTS {DB}
            CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci;
        """)

engine = create_engine(ENGINE_URL, future=True, pool_pre_ping=True)

In [9]:
DDL_ARTICLES_MYSQL = """
CREATE TABLE IF NOT EXISTS `articles` (
  `id` BIGINT UNSIGNED AUTO_INCREMENT PRIMARY KEY,
  `source` VARCHAR(255),
  `url` TEXT NOT NULL,
  `url_hash` CHAR(32) NOT NULL,
  `title` TEXT,
  `description` MEDIUMTEXT,
  `content` MEDIUMTEXT,
  `full_text` MEDIUMTEXT,
  `published_date` VARCHAR(32) NULL,    -- Date de publication de l'article
  `gdelt_date` VARCHAR(32) NULL,        -- Date de découverte par GDELT
  `language` VARCHAR(16),
  `sentiment_compound` DOUBLE,
  `sentiment_pos` DOUBLE,
  `sentiment_neu` DOUBLE,
  `sentiment_neg` DOUBLE,
  `sentiment_label` VARCHAR(16),
  UNIQUE KEY `uk_url_hash` (`url_hash`),
  KEY `idx_published_date` (`published_date`),
  KEY `idx_gdelt_date` (`gdelt_date`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_general_ci;
"""

def create_table_safely():
    """Crée la table articles de manière sécurisée avec vérifications"""
    try:
        with engine.begin() as conn:
            # Vérification si la table existe déjà
            if USE_MARIADB:
                result = conn.exec_driver_sql("""
                    SELECT COUNT(*) as count FROM information_schema.tables 
                    WHERE table_schema = %s AND table_name = 'articles'
                """, (DB,))
                exists = result.scalar() > 0
            else:
                result = conn.exec_driver_sql("""
                    SELECT COUNT(*) as count FROM sqlite_master 
                    WHERE type='table' AND name='articles'
                """)
                exists = result.scalar() > 0
            
            if exists:
                print(" Table 'articles' existe déjà")
            else:
                print(" Création de la table 'articles'...")
                conn.exec_driver_sql(DDL_ARTICLES_MYSQL if USE_MARIADB else DDL_ARTICLES_SQLITE)
                print(" Table 'articles' créée avec succès")
            
            # Vérification finale
            if USE_MARIADB:
                result = conn.exec_driver_sql("SHOW TABLES LIKE 'articles'")
                if result.rowcount == 0:
                    raise Exception("La table 'articles' n'a pas été créée correctement")
            else:
                result = conn.exec_driver_sql("SELECT name FROM sqlite_master WHERE type='table' AND name='articles'")
                if not result.fetchone():
                    raise Exception("La table 'articles' n'a pas été créée correctement")
                    
        print("Connexion et schéma OK :", ENGINE_URL)
        
    except Exception as e:
        print(f" Erreur lors de la création de la table: {e}")
        print("Tentative de création forcée...")
        
        # Tentative de création forcée
        try:
            with engine.begin() as conn:
                if USE_MARIADB:
                    conn.exec_driver_sql("DROP TABLE IF EXISTS articles")
                conn.exec_driver_sql(DDL_ARTICLES_MYSQL if USE_MARIADB else DDL_ARTICLES_SQLITE)
            print(" Table créée après suppression forcée")
        except Exception as e2:
            print(f" Impossible de créer la table: {e2}")
            raise

# Création de la table
create_table_safely()

 Erreur lors de la création de la table: name 'engine' is not defined
Tentative de création forcée...
 Impossible de créer la table: name 'engine' is not defined


NameError: name 'engine' is not defined

In [97]:
analyzer = SentimentIntensityAnalyzer()

def clean_text_soft(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = html.unescape(text)
    text = BeautifulSoup(text, "html.parser").get_text(" ", strip=True)
    text = re.sub(r'(https?://\S+|www\.\S+)', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def label_from_compound(x: float) -> str:
    return "Positive" if x >= 0.05 else ("Negative" if x <= -0.05 else "Neutral")

def generate_url_hash(url: str) -> str:
    """Génère un hash MD5 de l'URL pour la déduplication"""
    if not url:
        return ""
    return hashlib.md5(url.encode('utf-8')).hexdigest()

In [6]:
# ==== GDELT -> DF avec Multiple Batches ====================================
def get_multiple_batches(num_batches=1):
    gd = GdeltDoc()
    all_articles = []
    from datetime import datetime, timedelta
    import time
    
    start_date = datetime(2017, 1, 1)
    end_date = datetime(2025, 9, 20)
    total_days = (end_date - start_date).days
    days_per_batch = total_days // num_batches
    current_date = start_date
    
    for i in range(num_batches):
        if i == num_batches - 1:
            period_end = end_date
        else:
            period_end = current_date + timedelta(days=days_per_batch)
            
        f = Filters(
            start_date=current_date.strftime("%Y-%m-%d"),
            end_date=period_end.strftime("%Y-%m-%d"),
            num_records=250,
            language="ENGLISH",
            domain=[ "bloomberg.com", "theguardian.com", "ft.com","economist.com"]
        )
        
        try:
            df_batch = gd.article_search(f)
            if not df_batch.empty:
                all_articles.append(df_batch)
                print(f"Batch {i+1} ({current_date.strftime('%Y-%m-%d')} à {period_end.strftime('%Y-%m-%d')}): {len(df_batch)} articles")
        except Exception as e:
            print(f"Erreur batch {i+1}: {e}")
            
        current_date = period_end
        time.sleep(1)
    
    if all_articles:
        final_df = pd.concat(all_articles, ignore_index=True)
        final_df = final_df.drop_duplicates(subset=['url'], keep='first')
        print(f"Total final après suppression doublons: {len(final_df)} articles")
        return final_df
    return pd.DataFrame()



In [ ]:
# news = get_multiple_batches(num_batches=200)

# news = news[['title', 'seendate']]
# news.columns = ['title', 'date']
# # Changer de format de 20170117T183000Z = 
# news['date'] = pd.to_datetime(news['date'], format = '%Y%m%dT%H%M%SZ')
# news['hour'] = news['date'].dt.strftime('%H:%M:%S')
# news['date'] = news['date'].dt.date
# news['score'] = news['title'].apply(SentimentIntensityAnalyzer)

In [8]:
# ==== SCORING ================================================================
def sentiment_on_df(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        return df
        
    title   = df["title"]        if "title" in df.columns else pd.Series([""]*len(df))
    content = df["content"]      if "content" in df.columns else pd.Series([""]*len(df))
    desc    = df["description"]  if "description" in df.columns else pd.Series([""]*len(df))
    snip    = df["snippet"]      if "snippet" in df.columns else pd.Series([""]*len(df))
    lang    = df["language"]     if "language" in df.columns else pd.Series([""]*len(df))
    url     = df["url"]          if "url" in df.columns else df.get("DocumentIdentifier", pd.Series([""]*len(df)))
    source  = df["domain"]       if "domain" in df.columns else df.get("sourceCommonName", pd.Series([""]*len(df)))
    

    
    # Gestion séparée des deux types de dates
    published_date = None
    gdelt_date = None
    
    # Date de publication de l'article (priorité : publishdate > date)
    if "publishdate" in df.columns:
        published_date = df["publishdate"]
    elif "date" in df.columns:
        published_date = df["date"]
    else:
        published_date = pd.Series([None]*len(df))
    
    # Date de découverte par GDELT
    if "seendate" in df.columns:
        gdelt_date = df["seendate"]
    else:
        gdelt_date = pd.Series([None]*len(df))
    
    full_text = (title.fillna("") + " " + content.fillna("") + " " + desc.fillna("") + " " + snip.fillna("")).map(clean_text_soft)
    scores = full_text.map(lambda t: analyzer.polarity_scores(t) if t else {"compound":0,"pos":0,"neu":1,"neg":0})
    
    out = pd.DataFrame({
        "source":  source.astype(str).str[:255],
        "url":     url.astype(str).str[:1024],
        "url_hash": url.astype(str).map(generate_url_hash),
        "title":   title.astype(str),
        "description": desc.astype(str),
        "content": content.astype(str),
        "full_text": full_text,
        "language": lang.astype(str).str[:16],
        "published_date": published_date,  # Date de publication
        "gdelt_date": gdelt_date          # Date de découverte GDELT
    })
    
    out["sentiment_compound"] = scores.map(lambda s: s["compound"])
    out["sentiment_pos"]      = scores.map(lambda s: s["pos"])
    out["sentiment_neu"]      = scores.map(lambda s: s["neu"])
    out["sentiment_neg"]      = scores.map(lambda s: s["neg"])
    out["sentiment_label"]    = out["sentiment_compound"].map(label_from_compound)
    
    print(" COLONNES DATES ORIGINALES:")
    if published_date is not None and not published_date.empty and published_date.notna().any():
        print(f"published_date exemples: {published_date.dropna().head(3).tolist()}")
    if gdelt_date is not None and not gdelt_date.empty and gdelt_date.notna().any():
        print(f"gdelt_date exemples: {gdelt_date.dropna().head(3).tolist()}")
    
    print(f"Résultat final - published_date nulles: {out['published_date'].isna().sum()}/{len(out)}")
    print(f"Résultat final - gdelt_date nulles: {out['gdelt_date'].isna().sum()}/{len(out)}")


    return out


NameError: name 'out' is not defined

In [100]:
# ==== FONCTION DE NETTOYAGE DES DATES ======================================
def _to_sql_value_dt(x):
    """Convertit une valeur de date en string pour stockage SQL"""
    if x is None:
        return None
    try:
        import pandas as pd
        if pd.isna(x):
            return None
    except Exception:
        pass
    if isinstance(x, str):
        s = x.strip()
        if s == "" or s.lower() in ("none", "nan", "nat", "null"):
            return None
        return s
    return str(x)



In [ ]:
# Conversion du format des dates dans le DataFrame de sortie
out['published_date'] = pd.to_datetime(out['published_date'], errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')
out['gdelt_date'] = pd.to_datetime(out['gdelt_date'], errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')

In [101]:
# ==== UPSERT EN DB ===========================================================
def upsert_articles(df_scored: pd.DataFrame):
    if df_scored.empty:
        print("Aucun article à insérer.")
        return 0, 0

    # Vérification que la table existe avant insertion
    try:
        with engine.begin() as conn:
            if USE_MARIADB:
                result = conn.exec_driver_sql("SHOW TABLES LIKE 'articles'")
                if result.rowcount == 0:
                    raise Exception("Table 'articles' non trouvée")
            else:
                result = conn.exec_driver_sql("SELECT name FROM sqlite_master WHERE type='table' AND name='articles'")
                if not result.fetchone():
                    raise Exception("Table 'articles' non trouvée")
    except Exception as e:
        print(f"❌ Erreur: {e}")
        print("Recréation de la table...")
        create_table_safely()

    if USE_MARIADB:
        cols = ["source","url","url_hash","title","description","content","full_text",
                "published_date","gdelt_date","language",
                "sentiment_compound","sentiment_pos","sentiment_neu","sentiment_neg","sentiment_label"]
    else:
        cols = ["source","url","title","description","content","full_text",
                "published_date","gdelt_date","language",
                "sentiment_compound","sentiment_pos","sentiment_neu","sentiment_neg","sentiment_label"]

    for c in cols:
        if c not in df_scored.columns:
            df_scored[c] = None
    
        if "gdelt_date" in df_scored.columns:
            gd = df_scored["gdelt_date"].astype(str).str.extract(r"^(\d{8})")[0]
            df_scored["gdelt_date"] = gd.where(gd.notna() & (gd.str.len() == 8), None)


    payload = []
    for _, r in df_scored.iterrows():
        rec = {c: r.get(c) for c in cols}
        rec["published_date"] = _to_sql_value_dt(rec["published_date"])
        rec["gdelt_date"] = _to_sql_value_dt(rec["gdelt_date"])
        payload.append(rec)

    try:
        with engine.begin() as conn:
            if USE_MARIADB:
                sql = text("""
                INSERT INTO articles
                  (source, url, url_hash, title, description, content, full_text,
                   gdelt_date, language,
                   sentiment_compound, sentiment_pos, sentiment_neu, sentiment_neg, sentiment_label)
                VALUES
                  (:source, :url, :url_hash, :title, :description, :content, :full_text,
                   :gdelt_date, :language,
                   :sentiment_compound, :sentiment_pos, :sentiment_neu, :sentiment_neg, :sentiment_label)
                ON DUPLICATE KEY UPDATE
                  title=VALUES(title),
                  description=VALUES(description),
                  content=VALUES(content),
                  full_text=VALUES(full_text),
                  gdelt_date=VALUES(gdelt_date),
                  language=VALUES(language),
                  sentiment_compound=VALUES(sentiment_compound),
                  sentiment_pos=VALUES(sentiment_pos),
                  sentiment_neu=VALUES(sentiment_neu),
                  sentiment_neg=VALUES(sentiment_neg),
                  sentiment_label=VALUES(sentiment_label)
                """)
            else:
                sql = text("""
                INSERT INTO articles
                  (source, url, title, description, content, full_text,
                   gdelt_date, language,
                   sentiment_compound, sentiment_pos, sentiment_neu, sentiment_neg, sentiment_label)
                VALUES
                  (:source, :url, :title, :description, :content, :full_text,
                   :gdelt_date, :language,
                   :sentiment_compound, :sentiment_pos, :sentiment_neu, :sentiment_neg, :sentiment_label)
                ON CONFLICT(url) DO UPDATE SET
                  title=excluded.title,
                  description=excluded.description,
                  content=excluded.content,
                  full_text=excluded.full_text,
                  gdelt_date=excluded.gdelt_date,
                  language=excluded.language,
                  sentiment_compound=excluded.sentiment_compound,
                  sentiment_pos=excluded.sentiment_pos,
                  sentiment_neu=excluded.sentiment_neu,
                  sentiment_neg=excluded.sentiment_neg,
                  sentiment_label=excluded.sentiment_label
                """)

            conn.execute(sql, payload)

        print(f"✅ Écrit dans la base: {len(payload)} lignes (insert+update confondus).")
        return len(payload), 0
        
    except Exception as e:
        print(f"❌ Erreur lors de l'insertion: {e}")
        
        # En cas d'erreur, afficher quelques exemples de données pour debug
        print("🔍 Exemples de données à insérer:")
        for i, item in enumerate(payload[:3]):
            print(f"  Ligne {i+1}: {list(item.keys())}")
        raise

In [102]:
def reset_articles_table():
    """Vide la table articles (plus rapide que DROP/CREATE)"""
    with engine.begin() as conn:
        if USE_MARIADB:
            # Vide tout en conservant schéma + indexes + AUTO_INCREMENT
            conn.exec_driver_sql("TRUNCATE TABLE articles;")
        else:
            # SQLite n'a pas TRUNCATE
            conn.exec_driver_sql("DELETE FROM articles;")
            # Reset l'AUTOINCREMENT de SQLite
            try:
                conn.exec_driver_sql("DELETE FROM sqlite_sequence WHERE name='articles';")
            except Exception:
                pass

In [103]:
# ==== EXÉCUTION ==============================================================
print("Récupération des articles GDELT...")
df = get_multiple_batches(6)

if not df.empty:
    print("Analyse de sentiment...")
    scored = sentiment_on_df(df)

    # ---- Agrégation par jour + création de table sentiment_daily ---------------
    # 1) normaliser la date au format YYYYMMDD
    scored["gdelt_date"] = scored["gdelt_date"].astype(str).str.extract(r"^(\d{8})")[0]
    # 2) s'assurer que le sentiment est numérique
    scored["sentiment_compound"] = pd.to_numeric(scored["sentiment_compound"], errors="coerce")
    # 3) calcul de l’agrégat
    daily_sentiment = (
        scored.dropna(subset=["gdelt_date","sentiment_compound"])
            .groupby("gdelt_date", as_index=False)
            .agg(
                sentiment_moyen=("sentiment_compound","mean"),
                nb_articles=("sentiment_compound","size")
            )
            .sort_values("gdelt_date")
    )

    # 4) nettoyer tout objet existant et écrire la TABLE 'sentiment_daily'
    from sqlalchemy.types import CHAR, Float, Integer
    if USE_MARIADB:
        with engine.begin() as conn:
            conn.exec_driver_sql("DROP VIEW IF EXISTS sentiment_daily;")
            conn.exec_driver_sql("DROP TABLE IF EXISTS sentiment_daily;")

    daily_sentiment.to_sql(
        "sentiment_daily",
        engine,
        if_exists="replace",
        index=False,
        dtype={"gdelt_date": CHAR(8), "sentiment_moyen": Float, "nb_articles": Integer},
    )

    # 5) index (safe) pour accélérer les requêtes
    if USE_MARIADB:
        with engine.begin() as conn:
            try:
                conn.exec_driver_sql("ALTER TABLE sentiment_daily ADD INDEX idx_sd_date (gdelt_date);")
            except Exception:
                pass

    print("\n📈 sentiment_daily créée. Aperçu :")
    print(daily_sentiment.head(10))


    if not scored.empty:
        print("\nAperçu des résultats:")
        print(scored[["title","sentiment_label","sentiment_compound"]].head(10))
        
        print("Reset table articles...")
        reset_articles_table()

        print("\nInsertion en base de données...")
        upsert_articles(scored)
        print("Terminé avec succès :", ENGINE_URL)
    else:
        print("Aucun article après scoring.")
else:
    print("Aucun article récupéré depuis GDELT.")

Récupération des articles GDELT...


Batch 1 (2020-01-01 à 2020-12-14): 250 articles
Batch 2 (2020-12-14 à 2021-11-27): 250 articles
Batch 2 (2020-12-14 à 2021-11-27): 250 articles
Batch 3 (2021-11-27 à 2022-11-10): 250 articles
Batch 3 (2021-11-27 à 2022-11-10): 250 articles
Batch 4 (2022-11-10 à 2023-10-24): 250 articles
Batch 4 (2022-11-10 à 2023-10-24): 250 articles
Batch 5 (2023-10-24 à 2024-10-06): 250 articles
Batch 5 (2023-10-24 à 2024-10-06): 250 articles
Batch 6 (2024-10-06 à 2025-09-20): 250 articles
Batch 6 (2024-10-06 à 2025-09-20): 250 articles
Total final après suppression doublons: 1497 articles
Analyse de sentiment...
Total final après suppression doublons: 1497 articles
Analyse de sentiment...
🔍 COLONNES DATES ORIGINALES:
gdelt_date exemples: ['20200916T053000Z', '20200915T080000Z', '20200915T224500Z']
Résultat final - published_date nulles: 1500/1500
Résultat final - gdelt_date nulles: 3/1500

📈 sentiment_daily créée. Aperçu :
  gdelt_date  sentiment_moyen  nb_articles
0   20200915        -0.178693     

ProgrammingError: (pymysql.err.ProgrammingError) nan can not be used with MySQL
[SQL: 
                INSERT INTO articles
                  (source, url, url_hash, title, description, content, full_text,
                   gdelt_date, language,
                   sentiment_compound, sentiment_pos, sentiment_neu, sentiment_neg, sentiment_label)
                VALUES
                  (%(source)s, %(url)s, %(url_hash)s, %(title)s, %(description)s, %(content)s, %(full_text)s,
                   %(gdelt_date)s, %(language)s,
                   %(sentiment_compound)s, %(sentiment_pos)s, %(sentiment_neu)s, %(sentiment_neg)s, %(sentiment_label)s)
                ON DUPLICATE KEY UPDATE
                  title=VALUES(title),
                  description=VALUES(description),
                  content=VALUES(content),
                  full_text=VALUES(full_text),
                  gdelt_date=VALUES(gdelt_date),
                  language=VALUES(language),
                  sentiment_compoun